<a href="https://colab.research.google.com/github/pn0159/prathima_INFO5731_Fall2020/blob/master/Prathima_In_class_exercise_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The eighth in-class-exercise (20 points in total, 10/29/2020)**

The data for this exercise is from the dataset you created from assignment three. Please perform answer the following questions based on your data:

## (1) (10 points) Write a python program to extract the sentiment related terms from the corpus. You may use python package such as polyglot or external lexicon resources in the question. Rank the sentiment related terms by frequency.

In [ ]:
!pip install pytreebank
import pytreebank
import sys
import os
out_path = os.path.join(sys.path[0],'sst_{}.txt')
dataset = pytreebank.load_sst('./cleaned_data')
for category in ['train', 'test']:
    with open(out_path.format(category), 'w') as outfile:
        for item in dataset[category]:
            outfile.write("__label__{}\t{}\n".format(
                item.to_labeled_lines()[0][0] + 1,
                item.to_labeled_lines()[0][1]
            ))

print(len(dataset['train']))

8544


In [109]:
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from textblob import TextBlob

my1data = pd.read_csv("/content/sample_data/mydata.csv")
my1data = my1data[['Abstract']]
my1data['Abstract'] = my1data['Abstract'].str.replace('[^\w\s]','')
my1data['Abstract'] = my1data['Abstract'].apply(lambda x: " ".join(x.lower() for x in x.split()))
my1data['Abstract'] = my1data['Abstract'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
my1data['Abstract'] = my1data['Abstract'].apply(lambda x: nltk.word_tokenize(x))
mydata_freq = (my1data['Abstract']).apply(lambda x: pd.value_counts(x)).sum(axis = 0).reset_index()
mydata_freq.columns = ['List of Words', 'Value of tf']
mydata_freq['Value of polarity'] = mydata_freq['List of Words'].apply(lambda x: TextBlob(x).sentiment.polarity)
senti_terms = mydata_freq.loc[mydata_freq['Value of polarity'] != 0].sort_values(by='Value of tf', ascending=False)
senti_terms = senti_terms.reset_index(drop=True)
print('Ranking of sentiment related terms by frequency:')
senti_terms

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Ranking of sentiment related terms by frequency:


,List of Words,Value of tf,Value of polarity
0,natural,71.0,0.100000
1,linguistic,10.0,0.100000
2,new,8.0,0.136364
3,many,7.0,0.500000
4,effective,7.0,0.600000
...,...,...,...
85,cultural,1.0,0.100000
86,conventional,1.0,-0.142857
87,contemporary,1.0,0.166667
88,fit,1.0,0.400000


## (2) (10 points) Compare the performance of the following tools in sentiment identification: TextBlob (https://textblob.readthedocs.io/en/dev/), VADER (https://github.com/cjhutto/vaderSentiment), TFIDF-based Support Vector Machine (SVM) (Split your data into training and testing data). Take your own annotation as the standard answers. 

Reference code: https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4

In [54]:
import pandas as pd
df=pd.read_csv("/content/sample_data/cleaned_data.csv",usecols=["cleaned_text","Sentiment"])
df=df.dropna().reset_index()

In [55]:
df

,index,cleaned_text,Sentiment
0,0,introduction statistical natural language proc...,positive
1,1,the paper summarizes essential property docume...,positive
2,2,abstract language way communicating word langu...,positive
3,3,we report experiment use standard natural lang...,positive
4,4,paper describe simple rule based approach auto...,positive
...,...,...,...
88,95,bibliography,positive
89,96,applying natural language processing technique...,positive
90,97,example natural language chinese english itali...,positive
91,98,a number powerful modelling technique develope...,positive


In [85]:
from sklearn.metrics import f1_score, accuracy_score
textblob_data = pd.read_csv("/content/sample_data/myfinaldata.csv")
textblob_data['polarity'] = textblob_data['Abstract'].apply(lambda x: TextBlob(x).sentiment.polarity)
textblob_data['predicted sentiment'] = pd.cut(textblob_data['polarity'], bins=4, labels=[1, 2, 3, 4])
#Defining sentiment
def sentiment(x):
    if x in [1, 2]:
        return 'Negative'
    if x == 3:
        return 'Neutral'
    if x == 4:
        return 'Positive'
textblob_data['predicted sentiment'] = textblob_data['predicted sentiment'].apply(lambda x: sentiment(x))
print(textblob_data[['documentid', 'sentiment', 'predicted sentiment']].head(5))
textblob_accuracy = accuracy_score(textblob_data['sentiment'], textblob_data['predicted sentiment'])*100
textblob_f1 = f1_score(textblob_data['sentiment'], textblob_data['predicted sentiment'], average='macro')
print(' f1-score of the TextBlob:', textblob_f1)


   documentid sentiment predicted sentiment
0           0  positive             Neutral
1           1  positive             Neutral
2           2  positive             Neutral
3           3  positive             Neutral
4           4  positive             Neutral
 f1-score of the TextBlob: 0.0


In [87]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
vader = SentimentIntensityAnalyzer()

myvader_data = pd.read_csv("/content/sample_data/myfinaldata.csv")
myvader_data['polarity'] = textblob_data['Abstract'].apply(lambda x: vader.polarity_scores(x)['compound'])
myvader_data['predicted sentiment'] = pd.cut(myvader_data['polarity'], bins=4, labels=[1, 2, 3, 4])
myvader_data['predicted sentiment'] = myvader_data['predicted sentiment'].apply(lambda x: sentiment(x))
print(myvader_data[['documentid', 'sentiment', 'predicted sentiment']].head(5))
vader_accuracy = accuracy_score(myvader_data['sentiment'], myvader_data['predicted sentiment'])*100
vader_f1 = f1_score(myvader_data['sentiment'], myvader_data['predicted sentiment'], average='macro')
print(' f1-score of the VADER sentiment identification :', vader_f1)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
   documentid sentiment predicted sentiment
0           0  positive             Neutral
1           1  positive            Positive
2           2  positive            Positive
3           3  positive            Positive
4           4  positive            Negative
 f1-score of the VADER sentiment identification : 0.0


In [101]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

mysvm = pd.read_csv("/content/sample_data/myfinaldata.csv")
train, test = sklearn.model_selection.train_test_split(mysvm, train_size=0.6, test_size=0.1)
mypip = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=100, 
                                           learning_rate='optimal', tol=None))])

svm = mypip.fit(train['Abstract'], train['sentiment'])
test['predicted sentiment'] = svm.predict(test['Abstract'])
print(test[['documentid', 'sentiment', 'predicted sentiment']].head(5))

mysvm_acc = accuracy_score(test['sentiment'], test['predicted sentiment'])*100
mysvm_f1 = f1_score(test['sentiment'], test['predicted sentiment'], average='macro')

print('Accuracy of the TFIDF-based SVM:', mysvm_acc)
print('f1-score of the TFIDF-based SVM :', mysvm_f1)

    documentid sentiment predicted sentiment
70          70  positive            positive
96          96  positive            positive
58          58  positive            positive
15          15  positive            positive
0            0  positive            positive
Accuracy of the TFIDF-based SVM: 90.0
f1-score of the TFIDF-based SVM : 0.4736842105263158


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
"""
Sentiment Analysis can be done by using 
1.TextBlob
2.VADER 
3.TF-IDF based SVM. 

TextBlob:TextBlob is a popular Python library for processing textual data. It is built on top of NLTK, another popular Natural Language Processing toolbox for Python. 
TextBlob uses a sentiment lexicon (consisting of predefined words) to assign scores for each word, which are then averaged out to give an overall sentence sentiment score.

VADER: Valence Aware Dictionary and sEntiment Reasoner is another popular rule-based library for sentiment analysis.
Like TextBlob, it uses a sentiment lexicon that contains intensity measures for each word based on human-annotated labels. A key difference however, is that VADER was designed with a focus on social m
This means that it puts a lot of emphasis on rules that capture the essence of text typically seen on social media

SVM:TF-IDF is a numerical statistic that is intended to reflect how important a word is to a text document.
Support Vector Machine (SVM) is one of the most frequently used classification algorithms in text categorization.
It can deal with linear and non-linear data classification with the help of kernel function

The performance of the models can be measured using accuracy score and f1 score. 

Analysis:
From my analysis,TF-IDF Based SVM has given the best accuracy and f1 score, with 90% and 0.47 respectively,
which means out of the three models i have used for doing sentiment analysis.SVM is the best model for sentiment analysis. 
VADER and Textblob was not at all goodto identify. 
